In [13]:
import requests
import msal
import json
import webbrowser
import time
import urllib3
import os

# Desabilitar avisos de certificado inseguro
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Tenant ID da Seatrium
tenant_id = "8d348135-f7aa-4a4c-a71c-31df44c7cac3"

# URL do site
sharepoint_site = "https://seatrium.sharepoint.com/sites/P84-P85VendorManagement"
api_endpoint = f"{sharepoint_site}/_api/web"

# Configurar proxy se necessário (ajuste conforme seu ambiente)
proxies = {
    # 'http': 'http://proxy.petrobras.com.br:porta',
    # 'https': 'http://proxy.petrobras.com.br:porta'
}

def get_access_token_interactive():
    # Usando o ID de um aplicativo público da Microsoft
    client_id = "872cd9fa-d31f-45e0-9eab-6e460a02d1f1"  # ID do Visual Studio
    
    # Criar aplicativo MSAL
    app = msal.PublicClientApplication(
        client_id,
        authority=f"https://login.microsoftonline.com/{tenant_id}"
    )
    
    # Tentar obter token do cache
    accounts = app.get_accounts()
    if accounts:
        print("Usando conta em cache...")
        result = app.acquire_token_silent(["https://seatrium.sharepoint.com/.default"], account=accounts[0])
        if result:
            print("Token obtido do cache!")
            return result.get("access_token")
    
    # Se não houver token em cache, usar autenticação interativa
    print("\nPor favor, faça login com suas credenciais de acesso ao SharePoint da Seatrium.")
    print("Uma janela do navegador será aberta para autenticação.")
    
    # Iniciar autenticação interativa
    flow = app.initiate_device_flow(scopes=["https://seatrium.sharepoint.com/.default"])
    
    if "user_code" not in flow:
        print(f"Erro ao iniciar fluxo de autenticação: {flow.get('error_description')}")
        return None
    
    print(f"\n{flow['message']}")
    
    # Tentar abrir o navegador automaticamente
    try:
        webbrowser.open(flow['verification_uri'])
    except:
        pass  # Se não conseguir abrir o navegador, o usuário pode usar o link manualmente
        
    print("\nAguardando autenticação...")
    
    # Aguardar usuário autenticar
    result = app.acquire_token_by_device_flow(flow)
    
    if "access_token" in result:
        print("Token obtido com sucesso!")
        return result["access_token"]
    else:
        print(f"Erro ao obter token: {result.get('error_description')}")
        return None

# Função recursiva para listar documentos
def list_documents(folder_relative_url):
    token = get_access_token_interactive()
    if not token:
        return
        
    url = f"{api_endpoint}/GetFolderByServerRelativeUrl('/sites/P84-P85VendorManagement/{folder_relative_url}')/Files"
    headers = {
        "Accept": "application/json;odata=verbose",
        "Authorization": f"Bearer {token}"
    }
    
    try:
        # Usar proxies se configurados e desabilitar verificação de certificado
        response = requests.get(url, headers=headers, proxies=proxies if proxies else None, verify=False)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar pasta: {str(e)}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Status: {e.response.status_code}")
            print(f"Resposta: {e.response.text}")
        return
        
    try:
        data = response.json()
        
        # Processar arquivos
        print(f"\nArquivos na pasta '{folder_relative_url}':")
        if len(data["d"]["results"]) == 0:
            print("  (Nenhum arquivo encontrado)")
        else:
            for file in data["d"]["results"]:
                print(f"  - {file['Name']}")
        
        # Obter subpastas
        folders_url = f"{api_endpoint}/GetFolderByServerRelativeUrl('/sites/P84-P85VendorManagement/{folder_relative_url}')/Folders"
        folders_response = requests.get(folders_url, headers=headers, proxies=proxies if proxies else None, verify=False)
        folders_response.raise_for_status()
        folders_data = folders_response.json()
        
        # Processar subpastas recursivamente
        subfolders = folders_data["d"]["results"]
        if subfolders:
            print(f"\nSubpastas encontradas em '{folder_relative_url}':")
            for folder in subfolders:
                relative_path = folder["ServerRelativeUrl"].replace("/sites/P84-P85VendorManagement/", "")
                print(f"  > {folder['Name']}")
                list_documents(relative_path)
        
    except Exception as e:
        print(f"Erro ao processar dados: {str(e)}")

# Iniciar a partir da pasta específica
print("Iniciando leitura de documentos no SharePoint da Seatrium...")
list_documents("Shared Documents/General/01 - TOPSIDE/1. Packages")


Iniciando leitura de documentos no SharePoint da Seatrium...

Por favor, faça login com suas credenciais de acesso ao SharePoint da Seatrium.
Uma janela do navegador será aberta para autenticação.

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code N85WFJZTL to authenticate.

Aguardando autenticação...
Token obtido com sucesso!
Erro ao acessar pasta: 401 Client Error: Unauthorized for url: https://seatrium.sharepoint.com/sites/P84-P85VendorManagement/_api/web/GetFolderByServerRelativeUrl('/sites/P84-P85VendorManagement/Shared%20Documents/General/01%20-%20TOPSIDE/1.%20Packages')/Files
Status: 401
Resposta: {"error":"invalid_request","error_description":"App is not allowed to call SPO with user_impersonation scope"}


### Script para capturar o Tenant ID da Seatrium

In [10]:
import requests

def get_tenant_id_from_domain(domain):
    url = f"https://login.microsoftonline.com/{domain}/.well-known/openid-configuration"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        # O Tenant ID está contido na URL do token endpoint
        token_endpoint = data.get("token_endpoint", "")
        # Formato: https://login.microsoftonline.com/{tenant_id}/oauth2/token
        tenant_id = token_endpoint.split("/")[3]
        return tenant_id
    else:
        print(f"Erro ao obter tenant ID: {response.status_code}")
        return None

# Usando o domínio da Seatrium
tenant_id = get_tenant_id_from_domain("seatrium.com")
print(f"Tenant ID: {tenant_id}")


Tenant ID: 8d348135-f7aa-4a4c-a71c-31df44c7cac3
